# **Preproccessing**

### Importing Asset

Dalam tahapan ini kita akan melakukan pengimportan terhadap hal hal yang memang diperlukan. Seperti kita akan mengimport library uyang akan dibutuhkan. Dan juga kita akn melakukan import pada datset kita. Selain itu kita akan melihat sedikit tentang isi datasetnya.

In [33]:
# import library
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re
import string
import pandas as pd
import numpy as np

In [34]:
# mengambil dataset
df = pd.read_csv("dataset\INA_TweetsPPKM_Labeled_Pure.csv", sep="\t")

# menampilkan dataset
df.head()

,Date,User,Tweet,sentiment
0,2022-03-31 14:32:04+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
1,2022-03-31 09:26:00+00:00,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1
2,2022-03-31 05:02:34+00:00,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1
3,2022-03-30 14:23:10+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
4,2022-03-30 11:28:57+00:00,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1


**0 = positif, 1 = netral, 2 = negatif**

lalu kita juga akan menelusuri berapa banyak data pada setiap label

In [35]:
# untuk menghitung banyaknya data pada setiap label
df["sentiment"].value_counts()

sentiment
1    17706
2     3980
0     1958
Name: count, dtype: int64

dari pengamatan di atas, kita dapat melihat bahwa tweet yang memiliki label netral jauh lebih banyak yang disusul dengan tweet negatif. Maka langkah yang kita akan ambil nantinya adalah dengan menyeimbangkan lagi untuk banyaknya data pada setiap label

### Feature Selection

dalam tahapan ini kita akan melakukan pengambilan fitur yang dibutuhkan saja. Untuk kasus text mining kita perlu komentarnya dan labelnya saja. Komentar sebagai x dan label sebagai y. Selain itu kita juga akan menyeimbangkan banyaknya data pada setiap label

In [36]:
# unuk menghilangkan kolom yang tidak diperlukan
df.drop(columns=["Date", "User"], inplace=True)

df.head()

,Tweet,sentiment
0,Ketahui informasi pembagian #PPKM di wilayah J...,1
1,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1
2,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1
3,Ketahui informasi pembagian #PPKM di wilayah J...,1
4,Kementerian Agama menerbitkan Surat Edaran Nom...,1


lalu kita akan melakukan pengecilan size. Agar seimbang kita tentukan banyaknya data pada setiap label adalh 1900, mengambil dari nilai terendah pada jenis label. Tujuan dari penyeimbangan data adalah untuk menghindari terjadinya overfitting pada label tertentu (cenderung menebak pada label yang banyak)

In [37]:
# untuk mengambil data 1900 dari masing2 jenis label
df = pd.concat([df[df.sentiment==0][:1900], df[df.sentiment==1][:1900], df[df.sentiment==2][:1900]])\
    .sample(frac=1, random_state=134).reset_index(drop=True)

df


,Tweet,sentiment
0,"Tak Terima Kota Malang Masuk PPKM Level 2, Wal...",1
1,"PPKM Dicabut, Booking Hotel Tetap Nihil https:...",1
2,Inilah Sistem Kerja Baru 2022 Bagi ASN Selama ...,1
3,JKW : Pak LBP nanti umumkan #PPKM #diperpanjan...,2
4,KABAR PANDEMI - Berpergian Sehat Pada Saat Pan...,1
...,...,...
5695,"#PPKM LEVEL 1000 yes , membagongkan",2
5696,"PPKM level 4 diperpanjang, sampai kapan? Nanti...",2
5697,"Mendagri Tito Karnavian mengatakan, PPKM Level...",1
5698,Program gagal kok yaa dilanjutin terus #PPKM #...,2


### Missing Value

Pada tahapan ini kita akan memeriksa apakah data kita terdapat yang kosong atau tidak. Tujuannya adalah untuk menghindari terjadinya error pada saat modeling algoritma

In [38]:
# mencari data kosong jenis Nan
df.isna().sum()

Tweet        0
sentiment    0
dtype: int64

In [39]:
# mencari data kosong jenis Null
df.isnull().sum()

Tweet        0
sentiment    0
dtype: int64

berhubungan tidak ada data yang kosong maka kita bisa langsung lanjut

### Duplicated Data

Selanjutnya kita juga akan melihat apakah ada data yang terduplikasi tujuannya adalah untuk menghindari overfitting. Model akan belajar hal yang sama secara berulang pada data yang terduplikasi sehingga model cenderung menghapal dan lebih mengingat data yang terduplikasi

In [40]:
# untuk menampilkan data yang terduplikasi
df[df.duplicated()]

,Tweet,sentiment
898,#PPKM,2
1258,Pemerintah Siapkan 3 Skenario Antisipasi Lonja...,1
1681,#semangat #ppkm,0
1691,Pemerintah Siapkan 3 Skenario Antisipasi Lonja...,1
1898,#ghostparade #ghostparadecomic #tidaktakuthant...,0
2108,#PPKM,2
2189,Pemerintah klo bikin acara rame2 di bolehin ti...,2
2286,#semangat #ppkm,0
2315,Hasil Pengeluaran BULLSEYE\nHari Ini : Selasa ...,1
2431,Pasti ada yang siap dan nggak siap dengan perp...,2


di atas merupakan beberapa data yang terduplikasi, maka kita perlu menghilangkan saja untuk data yang terduplikasi ini

In [41]:
# untuk menghilangkan data yang terduplikasi
df.drop_duplicates(inplace=True)

# untuk menampilkan data yang terduplikasi
df[df.duplicated]

,Tweet,sentiment


### Train Test Split

### Case Folding

### Tokenize

### Stopword

### Stemming

# **Pembobotan Kata**

### Analize Frequency Word

### Word Frequency on Document

### Term Frequency

### Inverse Document Frequency

### TF - IDF

### Feature Selection

# **Modeling**

### Create Class 

### Training

### Testing

### Conclusion